In [ ]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from tqdm import tqdm

import sys

### Loading Data

In [ ]:
np.set_printoptions(threshold=sys.maxsize)
torch.set_printoptions(threshold=sys.maxsize)

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data_path, labels_path):
        self.labels = torch.from_numpy(np.fromfile("labels", sep=" ")[1::2]).float()
        self.filenames = np.fromfile("labels", sep=" ")[0::2].astype(int)
        self.data_path = data_path

        self.data = {}
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        if i in self.data:
            vector = self.data[i]
        else:
            vector = np.fromfile(f"{self.data_path}/{self.filenames[i]}", sep=" ")[2::3]
            self.data[i] = vector
        return torch.from_numpy(vector).float(), torch.reshape(self.labels[i], (1,))

In [ ]:
training_data = CustomImageDataset("Training", "labels")

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)

### Neural Network

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(122 * 151, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.linear_relu_stack(x)

In [ ]:
model = NeuralNetwork()

### Training

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backward propagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
learning_rate = 1e-3
batch_size = 8
epochs = 10

In [ ]:
loss_fn = nn.BCELoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
for t in range(epochs):
    print(f"Epoch {t + 1}\n----------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)